# Chapter 4) Divide-and-Conquer

## 4.1) The maximum-subarray problem

### 4.1-1)
Due to setting `sum = 0` within `FindMaxCrossingSubarray`, no greater value than zero is ever found, thus (0, 0, 0) is the response.

### 4.1-2 & 4.1-3)
Both the recursive and brute-force versions have been [implemented in Go](https://github.com/jad-b/backtoschool/blob/master/algo/find_max_subarray.go). Running Benchmarks, I see this:
```bash
BenchmarkFindMaxSubarray/size=2-4       30000000                49.2 ns/op
BenchmarkFindMaxSubarray/size=4-4       10000000               126 ns/op
BenchmarkFindMaxSubarray/size=8-4        5000000               289 ns/op
BenchmarkFindMaxSubarray/size=16-4       2000000               667 ns/op
BenchmarkFindMaxSubarray/size=32-4       1000000              1435 ns/op
...
BenchmarkBruteForceFindMaxSubarray/size=2-4             100000000               20.7 ns/op
BenchmarkBruteForceFindMaxSubarray/size=4-4             30000000                50.3 ns/op
BenchmarkBruteForceFindMaxSubarray/size=8-4             10000000               160 ns/op
BenchmarkBruteForceFindMaxSubarray/size=16-4             2000000               707 ns/op
BenchmarkBruteForceFindMaxSubarray/size=32-4              500000              2848 ns/op
BenchmarkBruteForceFindMaxSubarray/size=64-4              200000             10498 ns/op
```

The crossover point, $n_0$, occurs sometime between 8 and 16. Setting the recursive version to call the brute-force solution when `(hi-lo)+1 < 16` produces these changes:
```bash
BenchmarkFindMaxSubarray/size=2-4       50000000                34.8 ns/op
BenchmarkFindMaxSubarray/size=4-4       20000000                62.5 ns/op
BenchmarkFindMaxSubarray/size=8-4       10000000               169 ns/op
BenchmarkFindMaxSubarray/size=16-4       3000000               418 ns/op
...
BenchmarkBruteForceFindMaxSubarray/size=2-4             100000000               20.3 ns/op
BenchmarkBruteForceFindMaxSubarray/size=4-4             20000000                53.0 ns/op
BenchmarkBruteForceFindMaxSubarray/size=8-4             10000000               156 ns/op
BenchmarkBruteForceFindMaxSubarray/size=16-4             2000000               866 ns/op
```
The straight brute-force solution is still faster when $n < 16$, presumably due to slightly less overhead. Thus, $n_0$ doesn't change.

### 4.1-4
Empty subarrays could be implemented two different ways. The `if high == low` in `FIND-MAXIMUM-SUBARRAY` could return the max between the single value and the empty subset: `max((low, high A[low]), (-1, -1, []))`. Or, the empty subset could be added as a fourth case to the `if-elseif-else` control flow within the same algorithm that selects the maximal value.

### 4.1-5
Hint: Take max of B[j] vs. B[j-1]
#ref=https://en.wikipedia.org/wiki/Maximum_subarray_problem
#retry

## Section 2) Strassen's algorithm for matrix multiplication 

### 4.2-2)
```
STRASSENS-MATRIX-MULTIPLY(A,B)
    n = A.rows
    C = Matrix(n, n)
    if n == 1  # Size one matrices
        C[1][1] = A[1][1] * B[1][1]
    else: 
        ASQR, BSQR, CSQR = # Partition A, B, and C into n/2 x n/2 submatrices by index
        
        S = []  # S will hold ten matrix multiplications; 
        # Perform 10 matrix subtractions/additions, S
        S[1] = BSQR[1][2] - BSQR[1][2]  # 10(n^2/4) = O(n^2)
        S[2] = ASQR[1][1] + ASQR[1][2]
        S[3] = ASQR[2][1] + ASQR[2][2]
        S[4] = BSQR[2][1] - BSQR[1][1]
        S[5] = ASQR[1][1] + ASQR[2][2]
        S[6] = BSQR[1][1] + BSQR[2][2]
        S[7] = ASQR[1][2] - ASQR[2][2]
        S[8] = BSQR[2][1] + BSQR[2][2]
        S[9] = ASQR[1][1] - ASQR[2][1]
        S[10] = BSQR[1][1] + BSQR[1][2]
        
        P = []  # Perform 7 recursive matrix multiplications, P
        # 7T(n/2) 
        P[1] = STRASSENS-MATRIX-MULTIPLY(ASQR[1][1], S[1])
        P[2] = STRASSENS-MATRIX-MULTIPLY(S[2], BSQR[2][2])
        P[3] = STRASSENS-MATRIX-MULTIPLY(S[3], BSQR[1][1])
        P[4] = STRASSENS-MATRIX-MULTIPLY(ASQR[2][2], S[4])
        P[5] = STRASSENS-MATRIX-MULTIPLY(S[5], S[6])
        P[6] = STRASSENS-MATRIX-MULTIPLY(S[7], S[8])
        P[7] = STRASSENS-MATRIX-MULTIPLY(S[9], S[10])
        
        # Re-build C through combinations of P
        # O(n^2)
        CSQR[1][1] = P[5] + P[4] - P[2] + P[6]  # 8(n^2/4)
        CSQR[1][2] = P[1] + [2]
        CSQR[2][1] = P[5] + P[4]
        CSQR[2][2] = P[5] + P[1] - P[3] - P[7]
        
        return C
```
#time=19m

### 4.2-4
Key intuition: The problem describes the recurrence $T(n) = kT(\frac{n}{3}) + O(n^2)$. 

Using the master theorem, we'd need to show that $n^2 = O(n^{log_3\,k-\epsilon})$ to prove that the above recurrence is dominated by the cost of the leaves, but we'll short-circuit this by leveraging our knowledge of Strassen's algorithm being $O(n^{lg\,7})$, and make the assumption this algorithm has a runtime of $O(n^{log_3\,k})$. 

Re-reading with the information above, the problem wants us to find a $k$ for which $n^{log_3\,k} = o(n^{lg\,7})$.

$$
\begin{align}
n^{log_3\,k} &= o(n^{lg\,7}) \\
n^{log_3\,k} &< cn^{lg\,7}) \qquad &\text{Let } c=1 \\
n^{log_3\,k} &< n^{lg\,7}) \\
log_3\,k &= lg\,7 & \text{By something} \\
3^{log_3\,k} &= 3^{lg\,7} \\
k &\approx 21.85
\end{align}
$$

#ref=https://atekihcan.github.io/CLRS/E04.02-04/
#time=27m23s

### 4.2-6)
The key intuition: Treat the matrices as $k$-length vectors, with $n\ x\ n$ submatrices for elements.

In a litte more detail: Pick a number for $n$, like 4. Picture an $n\ x\ n$ matrix; call it $A$. A $kn\ x\ n$ matrix is the size of $k\ A$ matrices stacked vertically. Likewise, a $n\ x\ kn$ matrix is the size of $A$ repeated $k$ times horizontally.

Now, substitute $A_i$ for the actual $n\ x\ n$ matrix, so $kn\ x\ n = [A_1, A_2, \ldots, A_k]$, and $n\ x\ kn = [B_1, B_2, \ldots, B_k]$. As per normal vector/matrix multiplication, $A\ x\ B$ produces a $kn\ x\ kn$ matrix, with values ranging from $A_1B_1$ to $A_k B_k$. Thus, there are $k^2$ multiplications of $n\ x\ n$ matrices required, each of which requires $n^{lg\,7}$ time, for a bound of $\Theta(k^2n^{lg\,7})$.

When mutiplying $n\ x\ kn$ and $kn\ x\ n$ matrices, you can again picture it as multiplying two vectors; this time a row vector by a column vector, with submatrices for elements. A row-by-column vector multiplication, each of length $k$, requires $k$ multiplications and $k-1$ summations between the products, for a bound of $\Theta(kn^{lg\,7})$.

#source=https://www.cs.helsinki.fi/webfm_send/1444

## 4.3) The substitution method for solving recurrences

### 4.3-1
$$
\begin{align}
T(n) &= T(n-1) + n \\
&= \Theta(n^2) \\
&= c(n-1)^2 + n \\
&= c(n^2 -2n +1) +n \\
&= cn^2 - 2cn + cn \\
&\le cn^2
\end{align}
$$
$T(1) = c1^2;\ \forall c \ge 1 \therefore$ the boundary condition holds.

### 4.3-4
$$
\begin{align}
T(n) &= 2T(\lfloor \frac{n}{2} \rfloor) + n; \quad \text{Let } T(n) = O(nlg\,n + n) \\
&\le 2(c\lfloor \frac{n}{2} \rfloor lg\,\lfloor \frac{n}{2} \rfloor + \lfloor \frac{n}{2} \rfloor) + n \\
&\le 2(c\frac{n}{2} lg\,\frac{n}{2} + \frac{n}{2}) + n \\
&= 2c\frac{n}{2} lg\,\frac{n}{2} + 2\frac{n}{2} + n \\
&= cn(lg\,n - lg\,2) + n + n \\
&= cn(lg\,n - 1) + 2n \\
&= cnlg\,n - cn + 2n \\
&= cnlg\,n - (2-c)n
\end{align}
$$
$T(1) = 1 \le c1lg\,1 + (2-c)1 = 0 + 2 - c = 1$ if $c = 1$

#source=http://clrs.skanev.com/04/03/04.html

### 4.3-5
Assert $T(n) \le cnlg\,n - d$.

$$
\begin{align}
T(n) &\le c\lceil\frac{n}{2}\rceil\,\lceil\frac{n}{2}\rceil - d +
    c\lfloor\frac{n}{2}\rfloor\,\lfloor\frac{n}{2}\rfloor - d + jn \\
&= c\frac{n}{2}lg\,\frac{n}{2} -d + c\frac{n}{2}\lg\,\frac{n}{2} -d + jn \\
&= cnlg\,\frac{n}{2} - 2d + jn \\
&= cn(lg\,n - lg\,2) - 2d + jn \\
&= cnlg\,n - cn - 2d + jn \\
&= cnlg\,n + (j-c)n - 2d \\
\\
cnlg\,n &\ge jn - cn - 2d \\
lg\,n &\ge \frac{j}{c} - 1 - \frac{2d}{cn}
\end{align}
$$

When $c >> j$, we see the last inequality go to $0-1-0$, satisfying $T(n) \le cnlg\,n - d$ as long as $n_0 = lg\,n > 1$. Likewise, the opposite holds true when $c << j$. 

## 4.4) The recursion-tree method for solving recurrences

### 4.4-1) 
Number of levels = $lg\,n + 1$. Subproblems per level = $3^i$. Size of subproblem at level $i = \frac{n}{2^i}$. Cost per level $= (\frac{3}{2})^i n$. The bottom level, or where $i = lg\, n$, can be analyzed to have $3^{lg\,n} = n^{lg\,3}$ nodes of cost 1. 

$$
\begin{align}
T(n) &= 3T(\frac{n}{2}) + n \\
&= n + (\frac{3}{2})^1 n + \ldots + (\frac{3}{2})^{n-1} n + \Theta(n^{lg\,3}) \\
&= \sum_{i=0}^{lg\,n-1} (\frac{3}{2})^in + \Theta(n^{lg\,3}) \\
&\le \sum_{i=0}^{\infty} (\frac{3}{2})^in + \Theta(n^{lg\,3}) \qquad \text{By equation A.6} \\
&= \frac{1}{1-(3/2)}n + \Theta(n^{lg\,3}) \\
&= \Theta(n^{lg\,3}) - 2n \\
& = O(n^{lg\,3})
\end{align}
$$

## 4.6) The Master Theorem

### 4.6-1
When $b$ is a positive integer, we can re-create 4.27 by the rule $n_j = \lceil \frac{n}{b^j} \rceil$.

#verified=http://math.stackexchange.com/a/510539/383092

## Problems

### 4-1
Written out; needs to copied into notebook. 

### 4-2
RecursiveBinarySearch

a) RecursiveBinarySearch = $T(n) = T(\frac{n}{2}) + \Theta(1)$.

Pointer) $T(\frac{n}{2}) + \Theta(1) = O(lg\,n)$

Copy) $T(\frac{n}{2}) + \Theta(N) + \Theta(1) = O(N lg\, N)$. We copy the entire array $lg\,N$ times.

Subrange) $T(\frac{n}{2}) + \Theta(n) + \Theta(1) = O(n)$.

b) MergeSort = $T(n) = 2T(\frac{n}{2}) + n$.

Pointer) $2T(\frac{n}{2}) + \Theta(1) + n = O(n\ lg\,n)$. No change.

Copy) $2T(\frac{n}{2}) + \Theta(N) + n = O(n^2)$.

Subrange) $2T(\frac{n}{2}) + \Theta(n) + n = O(2n\ lg\,n) = O(n\ lg\,n)$.

#redo